In [1]:
from src.data import _load_readmission_dataset, _load_simData
from src.helper_functions import _prepare_rt_tensors
from src.models import *
from src.losses import *
from src.training import *
from src.metrics import *

In [2]:
# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
x, t, e, d = _load_readmission_dataset(sequential=True)

/home/ayush/DeepReEvent/src/data.py:47: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  data['max_time'] = data.groupby('id')['t.stop'].transform(max)


In [4]:
# x, t, e, d = _load_simData(sequential=True)

In [5]:
readmission_tensor = _prepare_rt_tensors(x, t, e, d)
locals().update(readmission_tensor) # create variables from dictionary

/home/ayush/DeepReEvent/src/helper_functions.py:83: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  x = torch.tensor(x, dtype=torch.float32)


In [6]:
# model and training parameters

input_size = len(x[0]) 
output_size = int(max(t))
hidden_size = 8   # Number of units in the RNN layer

model = SimpleGRU(input_size, hidden_size, output_size, 2)

# Instantiate the model

num_epochs = 10000
patience = 3  # Number of epochs to wait for improvement before stopping
best_val_loss = float('inf')
wait = 0
loss_function = recurrent_terminal_loss

model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(),  lr=1e-4, weight_decay=1e-7)

In [7]:
#prepare data loaders

train_dataset = torch.utils.data.TensorDataset(x_train, t_train, e_train, d_train)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32)

val_dataset = torch.utils.data.TensorDataset(x_val, t_val, e_val, d_val)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=32)

In [8]:
train_validate_rt_model(model, train_dataloader, val_dataloader, loss_function, optimizer, num_epochs, patience, print_every=10)

Epoch: 0   Training loss: 4.9982   Validation loss: 4.8316


Epoch: 10   Training loss: 4.8899   Validation loss: 4.7345
Epoch: 20   Training loss: 4.8051   Validation loss: 4.6464
Epoch: 30   Training loss: 4.7237   Validation loss: 4.5641
Epoch: 40   Training loss: 4.6392   Validation loss: 4.4858
Epoch: 50   Training loss: 4.5624   Validation loss: 4.4102
Epoch: 60   Training loss: 4.4763   Validation loss: 4.3339
Epoch: 70   Training loss: 4.4100   Validation loss: 4.2557
Epoch: 80   Training loss: 4.3129   Validation loss: 4.1772
Epoch: 90   Training loss: 4.2480   Validation loss: 4.0972
Epoch: 100   Training loss: 4.1285   Validation loss: 4.0170
Epoch: 110   Training loss: 4.1035   Validation loss: 3.9377
Epoch: 120   Training loss: 4.0238   Validation loss: 3.8602
Epoch: 130   Training loss: 3.9522   Validation loss: 3.7875
Epoch: 140   Training loss: 3.8332   Validation loss: 3.7194
Epoch: 150   Training loss: 3.8453   Validation loss: 3.6553
Epoch: 160   Training loss: 3.7443   Validation loss: 3.5987
Epoch: 170   Training loss: 3.715

In [9]:
test_predictions = model(x_test).squeeze(-1)

survival_predictions = test_predictions[:, :, 0:1].squeeze(-1)
recurrent_predictions = test_predictions[:, :, 1:2].squeeze(-1)

In [10]:
calculate_survival_metrics(survival_predictions, d_train, t_train, d_test, t_test, d_val, t_val)

{'Brier Score': array([0.04007401, 0.03085015, 0.02573729, 0.01930871]),
 '25th Quantile CI': 0.9513157894736842,
 '50th Quantile CI': 0.9488103527124182,
 '75th Quantile CI': 0.9488103527124182}

In [11]:
import torch
recurrent_cindex(recurrent_predictions, e_test, t_test, 12)

[0.5788299454635597,
 0.641725352112676,
 0.7029662738724096,
 0.6242998707453684,
 0.5429951690821256,
 0.5429951690821256,
 0.5429951690821256,
 0.5429951690821256,
 0.5429951690821256,
 0.5429951690821256]